# Group Project

## Data Preprocess

**This is to preprocess the data scraped from the website and create metadata csv file. One of the result is a csv file containing lists to diaplay tags or articles at the same webpage, which is convinient for human to read. Another one dosen't contain lists, where all the content exist just as strings.This is suitable for computational analyzing.**

by:zhuoli Lu




## Import Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Installing and Importing

In [2]:
# Install and import spacy and plotly.
%pip install spaCy
%pip install plotly
%pip install nbformat --upgrade

In [3]:
# Import spacy
import spacy

# Import os to upload documents and metadata
import os

import yaml

# Import json
import json

# Load spaCy visualizer
from spacy import displacy

# Import pandas DataFrame packages
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

# Import graphing package
import plotly.graph_objects as go
import plotly.express as px

## Define a function to extract JSON onjects

In [4]:
def extract_json_objects(text, delimiter=','):

    # Create an empty list to store individual JSON objects,track the depth of braces and starting position of a JSON object
    objs = []
    depth = 0
    start = 0
    # Iterate over each character in the text
    for i, char in enumerate(text):
        if char == '{':
            if depth == 0:
                start = i  # Start of a new JSON object
            depth += 1 # Increase depth for each opening brace
        elif char == '}':
            depth -= 1   # Decrease depth for each closing brace
            if depth == 0:
                # to find a complete JSON object
                try:
                # Try to parse the JSON object and add it to the list
                    objs.append(json.loads(text[start:i+1]))
                except json.JSONDecodeError as e:
                  # Print an error message if JSON is not properly formatted
                    print(f"Error parsing JSON: {e}")
        elif char == delimiter and depth == 0:
           # Ignore the top-level separator (e.g., comma between JSON objects)
            continue
    # Return the list of parsed JSON objects
    return objs


## Create a DataFrame from parsing JSON strings in all files

In [5]:


# Create empty list for storing data
all_data = []

# Specify the directory where the files are located
folder_path = '/content/drive/MyDrive/Guardian_corpus_txt_improved'

# Iterate through each file in the folder
for file_name in os.listdir(folder_path):
    # Construct the full file path
    file_path = os.path.join(folder_path, file_name)

    # Make sure to process txt files only
    if os.path.isfile(file_path) and file_path.endswith('.txt'):
        # Read txt files
        with open(file_path, 'r', encoding='utf-8') as file:
          file_content = file.read()

        # Use yaml to parse JSON data
        try:
            data = yaml.safe_load(file_content)
            all_data.append(data)
        except yaml.YAMLError as e:
            print(f"Error parsing JSON in {file_name}: {e}")

# Transform the data into Pandas DataFrame
df = pd.DataFrame(all_data)

# Look at the head of the DataFrame
print(f"Total files processed: {len(all_data)}")
df.head()


Total files processed: 1200


,type,webTitle,sectionName,webPublicationDate,webUrl,tags,tagCount,bodyTextSummary
0,article,Volodymyr Zelenskiy stands defiant in face of ...,World news,2022-02-26T13:52:35Z,https://www.theguardian.com/world/2022/feb/26/...,"[{'tagTitle': 'Ukraine', 'tagURL': 'https://ww...",14,He looked gaunt and unshaven after another sle...
1,article,Record Covid cases in Russia and Ukraine compl...,World news,2022-02-04T15:06:11Z,https://www.theguardian.com/world/2022/feb/04/...,"[{'tagTitle': 'Ukraine', 'tagURL': 'https://ww...",13,The Omicron variant is causing record numbers ...
2,article,Back in the USSR: Lenin statues and Soviet fla...,World news,2022-04-23T16:01:52Z,https://www.theguardian.com/world/2022/apr/23/...,"[{'tagTitle': 'Ukraine', 'tagURL': 'https://ww...",11,Last week a familiar figure returned to the ma...
3,article,The artists of Ukraine find their voice in a c...,World news,2022-04-23T07:00:47Z,https://www.theguardian.com/world/2022/apr/23/...,"[{'tagTitle': 'Ukraine', 'tagURL': 'https://ww...",15,More than 200 years after Francisco Goya comme...
4,article,Rail staff killed in ‘unprecedented’ attack on...,World news,2022-03-29T10:42:05Z,https://www.theguardian.com/world/2022/mar/29/...,"[{'tagTitle': 'Nigeria', 'tagURL': 'https://ww...",10,Gunmen have attacked a train travelling from t...


In [17]:
import pandas as pd

# Assuming you have an original DataFrame named df
articles = {}  # Create a dictionary to store title, publication time, and summary for each article

# Iterate through the original DataFrame rows
for index, row in df.iterrows():
    # Extract article title and publication time
    title = row['webTitle']  # Assuming the article title is in the 'webTitle' column


    # Check if 'bodyTextSummary' exists in the row
    if 'bodyTextSummary' in row:
        summary = row['bodyTextSummary']

        # Initialize the article in the dictionary if it's new
        if title not in articles:
            articles[title] = {'Text': summary}

# Create lists for the new DataFrame
web_titles = []  # Store article titles
body_text_summaries = []  # Store article summaries

# Extract data from the articles dictionary
for title, info in articles.items():
    web_titles.append(title)
    body_text_summaries.append(info['Text'])  # This is now the summary for each article

# Create a new DataFrame
d1 = {'WebTitle': web_titles, 'Text': body_text_summaries}
title_text_df = pd.DataFrame(d1)

# Display the first few rows of the new DataFrame
title_text_df.head()



,WebTitle,Text
0,Volodymyr Zelenskiy stands defiant in face of ...,He looked gaunt and unshaven after another sle...
1,Record Covid cases in Russia and Ukraine compl...,The Omicron variant is causing record numbers ...
2,Back in the USSR: Lenin statues and Soviet fla...,Last week a familiar figure returned to the ma...
3,The artists of Ukraine find their voice in a c...,More than 200 years after Francisco Goya comme...
4,Rail staff killed in ‘unprecedented’ attack on...,Gunmen have attacked a train travelling from t...


## Parsing every elements, extracting all the articles, putting articles in the same webpage in a row, and spliting them by ",". Creating a dataframe.

In [18]:
# Assuming you have an original DataFrame named original_df
# Used to store extracted data
web_titles = []
pub_time = []
web_tags = []
tag_count = []
web_url = []


# Iterate through the original DataFrame
for index, row in df.iterrows():
    # Extract the title
    title = row['webTitle']  # Assuming the article title is in the 'webTitle' column
    time = row ['webPublicationDate']
    count = row['tagCount']
    url = row['webUrl']


    temp_tags = []



    # Examine whether the web has been processed
    if len(web_titles) > 0 and title == web_titles[-1] and time == pub_time[-1]:
        # if yes, add the new tag as the last tag in one raw
        if isinstance(row['tags'], list):
            for tag in row['tags']:
                if 'tagTitle' in tag:
                    web_tags[-1] += ", " + tag['tagTitle']  # Append the tag to the last item
    else:
        # if else, then process it as a new web in a new raw
        temp_tags = [tag['tagTitle'] for tag in row['tags'] if 'tagTitle' in tag] if isinstance(row['tags'], list) else []
        web_titles.append(title)
        pub_time.append(time)
        tag_count.append(count)
        web_url.append(url)
        web_tags.append(', '.join(temp_tags))  # Convert the list of tags to a comma-separated string


# Create a new DataFrame
d3 = {'WebTitle': web_titles, 'WebUrl' : web_url, 'PubTime':pub_time, 'Tags':web_tags, 'TagCounts':tag_count}
title_tag_df = pd.DataFrame(d3)

# Display the first few rows of the new DataFrame
title_tag_df.head()

,WebTitle,WebUrl,PubTime,Tags,TagCounts
0,Volodymyr Zelenskiy stands defiant in face of ...,https://www.theguardian.com/world/2022/feb/26/...,2022-02-26T13:52:35Z,"Ukraine, Volodymyr Zelenskiy, Europe, World ne...",14
1,Record Covid cases in Russia and Ukraine compl...,https://www.theguardian.com/world/2022/feb/04/...,2022-02-04T15:06:11Z,"Ukraine, Russia, Coronavirus, Europe, Infectio...",13
2,Back in the USSR: Lenin statues and Soviet fla...,https://www.theguardian.com/world/2022/apr/23/...,2022-04-23T16:01:52Z,"Ukraine, Russia, Europe, World news, Article, ...",11
3,The artists of Ukraine find their voice in a c...,https://www.theguardian.com/world/2022/apr/23/...,2022-04-23T07:00:47Z,"Ukraine, Art and design, Culture, World news, ...",15
4,Rail staff killed in ‘unprecedented’ attack on...,https://www.theguardian.com/world/2022/mar/29/...,2022-03-29T10:42:05Z,"Nigeria, Africa, World news, Article, News, Em...",10


## Merging the two dataframe above, creating a metadata csv file. Tags and articles in the same webpage exist in one row, splited by ",".

In [19]:
metadata_df = title_tag_df.merge(title_text_df,on='WebTitle')


## Transforming the time format to a datatime format, which is readable for machine in python

In [20]:
metadata_df['PubTime'] = pd.to_datetime(metadata_df['PubTime'])

metadata_df.head()

,WebTitle,WebUrl,PubTime,Tags,TagCounts,Text
0,Volodymyr Zelenskiy stands defiant in face of ...,https://www.theguardian.com/world/2022/feb/26/...,2022-02-26 13:52:35+00:00,"Ukraine, Volodymyr Zelenskiy, Europe, World ne...",14,He looked gaunt and unshaven after another sle...
1,Record Covid cases in Russia and Ukraine compl...,https://www.theguardian.com/world/2022/feb/04/...,2022-02-04 15:06:11+00:00,"Ukraine, Russia, Coronavirus, Europe, Infectio...",13,The Omicron variant is causing record numbers ...
2,Back in the USSR: Lenin statues and Soviet fla...,https://www.theguardian.com/world/2022/apr/23/...,2022-04-23 16:01:52+00:00,"Ukraine, Russia, Europe, World news, Article, ...",11,Last week a familiar figure returned to the ma...
3,The artists of Ukraine find their voice in a c...,https://www.theguardian.com/world/2022/apr/23/...,2022-04-23 07:00:47+00:00,"Ukraine, Art and design, Culture, World news, ...",15,More than 200 years after Francisco Goya comme...
4,Rail staff killed in ‘unprecedented’ attack on...,https://www.theguardian.com/world/2022/mar/29/...,2022-03-29 10:42:05+00:00,"Nigeria, Africa, World news, Article, News, Em...",10,Gunmen have attacked a train travelling from t...


## The beginnings of some of the texts may contain extra spaces (indicated by \t or \n). These characters can be replaced by a single space using the str.replace() method.

In [21]:
# Remove extra spaces from papers
metadata_df['Text'] = metadata_df['Text'].str.replace('\s+', ' ', regex=True).str.strip()
metadata_df.head()


,WebTitle,WebUrl,PubTime,Tags,TagCounts,Text
0,Volodymyr Zelenskiy stands defiant in face of ...,https://www.theguardian.com/world/2022/feb/26/...,2022-02-26 13:52:35+00:00,"Ukraine, Volodymyr Zelenskiy, Europe, World ne...",14,He looked gaunt and unshaven after another sle...
1,Record Covid cases in Russia and Ukraine compl...,https://www.theguardian.com/world/2022/feb/04/...,2022-02-04 15:06:11+00:00,"Ukraine, Russia, Coronavirus, Europe, Infectio...",13,The Omicron variant is causing record numbers ...
2,Back in the USSR: Lenin statues and Soviet fla...,https://www.theguardian.com/world/2022/apr/23/...,2022-04-23 16:01:52+00:00,"Ukraine, Russia, Europe, World news, Article, ...",11,Last week a familiar figure returned to the ma...
3,The artists of Ukraine find their voice in a c...,https://www.theguardian.com/world/2022/apr/23/...,2022-04-23 07:00:47+00:00,"Ukraine, Art and design, Culture, World news, ...",15,More than 200 years after Francisco Goya comme...
4,Rail staff killed in ‘unprecedented’ attack on...,https://www.theguardian.com/world/2022/mar/29/...,2022-03-29 10:42:05+00:00,"Nigeria, Africa, World news, Article, News, Em...",10,Gunmen have attacked a train travelling from t...


## Saving the metadata csv files.

In [23]:

metadata_df.to_csv('metadata_improved.csv')